### load specific sections, apply preprocessing, extract patches and save on disk

In [1]:
import os
import numpy as np
from tqdm import trange
from patch_extractor import PatchExtractor as PE
from utils import normalize_only
from keras_data import _compute_edges
from tensorflow.keras.utils import to_categorical

In [2]:
CLASS_NAMES = ["Basement", "SlopeMudA", "Deposit", "SlopeMudB", "SlopeValley", "Canyon"]
num_classes = len(CLASS_NAMES)

In [3]:
dataset = "/nas/public/dataset/geophysics/BlindTestSEG/"
out_root = "/nas/home/fpicetti/datasets/seg_blind_test"
os.makedirs(out_root, exist_ok=True)

### Train Dataset

In [4]:
image = np.load(os.path.join(dataset, "train_image.npy"))
label = np.load(os.path.join(dataset, "train_labels.npy")).astype(np.uint8)-1

In [5]:
print('x\ty\tz')
print('%d\t%d\t%d' % (image.shape[0],image.shape[1],image.shape[2]))

x	y	z
590	782	1006


In [6]:
normalize = lambda x: normalize_only(x, image.min(), image.max(), True)

#### vertical $992\times128$ patches

In [7]:
pe = PE(dim=(992,128), stride=(2,32))

Saving patches extracted from XZ sections

In [8]:
out_path = os.path.join(out_root, "normalized_categorical", "_".join([str(_) for _ in pe.dim+pe.stride]))
os.makedirs(out_path, exist_ok=True)

In [9]:
for i in trange(image.shape[1], ncols=90, desc="Inline section"):
    patches_img = normalize(pe.extract(image[:,i].T).reshape((-1,)+pe.dim+(1,)))
    patches_msk = to_categorical(pe.extract(label[:,i].T).reshape((-1,)+pe.dim+(1,)))
    patches_edg = _compute_edges(patches_msk)
    
    for p in range(patches_img.shape[0]):
        outname = os.path.join(out_path, "XZ%s_p%s.npy" % (str(i).zfill(3), str(p).zfill(len(str(patches_img.shape[0])))))
        
        np.save(outname, dict(img=patches_img[p], msk=patches_msk[p]), allow_pickle=True)


Inline section: 100%|███████████████████████████████████| 782/782 [31:26<00:00,  2.41s/it]


In [10]:
for i in trange(image.shape[0], ncols=90, desc="Crossline section"):
    patches_img = normalize(pe.extract(image[i].T).reshape((-1,)+pe.dim+(1,)))
    patches_msk = to_categorical(pe.extract(label[i].T).reshape((-1,)+pe.dim+(1,)))
    patches_edg = _compute_edges(patches_msk)
    
    for p in range(patches_img.shape[0]):
        outname = os.path.join(out_path, "YZ%s_p%s.npy" % (str(i).zfill(3), str(p).zfill(len(str(patches_img.shape[0])))))
        
        np.save(outname, dict(img=patches_img[p], msk=patches_msk[p]), allow_pickle=True)


Crossline section: 100%|████████████████████████████████| 590/590 [32:40<00:00,  3.32s/it]
